# Лабораторная работа №7. "Полносвязные нейронные сети (многослойный персептрон). Решение задач регрессии и классификации"

## Задание №1.
Решить задачи регрессии и классификации на данных в соответствии с Вашим индивидуальным вариантом (см. Лаб.работы №3, 4), используя полносвязные НС; реализовать НС посредством API Keras и фреймворка TensorFlow; оценить качество полученных моделей с помощью метрик.

In [2]:
from pprint import pprint
import warnings

from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
import tensorflow as tf

warnings.filterwarnings('ignore')

In [3]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

def metrics_c(actual, pred):
  return {
    'Accuracy': accuracy_score(actual, pred),
    'Precision': precision_score(actual, pred),
    'Recall': recall_score(actual, pred),
    'F1-score': f1_score(actual, pred),
    'ROC_AUC': roc_auc_score(actual, pred)
  }
models_c = {}

In [4]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
from math import sqrt

def metrics_r(actual, pred):
  return {
    'MAE': mean_absolute_error(actual, pred),
    'MSE': mean_squared_error(actual, pred),
    'RMSE': sqrt(mean_squared_error(actual, pred)),
    'MAPE': mean_absolute_percentage_error(actual, pred),
    'R^2': r2_score(actual, pred)
  }
models_r = {}

In [5]:
def best_model(models, metrics):
    return max([(val[metrics], k) for k, val in models.items()])

In [6]:
dfc = pd.read_csv('../data/neo_task_upd.csv')[:5561]
yc = dfc['hazardous']
Xc = dfc.drop(['hazardous'], axis=1)

smote = SMOTE()
Xc, yc = smote.fit_resample(Xc, yc)

scaler = MinMaxScaler()
Xc = scaler.fit_transform(Xc)

In [7]:
dfr = pd.read_csv('../data/energy_task_upd.csv', parse_dates=['date']).set_index('date')
yr = dfr['Appliances']
Xr = dfr.drop(['Appliances'], axis=1)

In [8]:
from sklearn.model_selection import train_test_split

Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xc, yc, test_size=0.2)
Xr_train, Xr_test, yr_train, yr_test = train_test_split(Xr, yr, test_size=0.2)

In [9]:
Xr.shape, Xc.shape

((9880, 28), (9980, 5))

### Регрессия

In [30]:
# создаем модель, как набор последовательных слоев
model_regression = tf.keras.Sequential(
    [
        # Dense - полносвязный слой (каждый нейрон следующего слоя связан со всеми нейронами предыдущего)
        tf.keras.layers.Dense(64, activation="relu", input_shape=(28,)),
        # на втором скрытом слое будет 32 нейрона
        tf.keras.layers.Dense(32, activation="linear"),
        # Dropout позволяет внести фактор случайности - при обучении часть нейронов будет отключаться
        # каждый нейрон, в данном случае, будет отключаться с вероятностью 0.1
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dropout(0.1),
        # на выходе один нейрон, функция активации не применяется
        tf.keras.layers.Dense(1, activation="linear"),
    ]
)

model_regression.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_8 (Dense)             (None, 64)                1856      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dropout_4 (Dropout)         (None, 32)                0         
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_11 (Dense)            (None, 1)                 17        
                                                                 
Total params: 4481 (17.50 KB)
Trainable params: 4481 (

In [40]:
model_regression.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="mse")

model_regression.fit(Xr_train, yr_train, epochs=120)

Epoch 1/120
247/247 [==============================] - 1s 2ms/step - loss: 463.0672
Epoch 2/120
247/247 [==============================] - 0s 2ms/step - loss: 457.9760
Epoch 3/120
247/247 [==============================] - 0s 2ms/step - loss: 452.6210
Epoch 4/120
247/247 [==============================] - 0s 2ms/step - loss: 463.2468
Epoch 5/120
247/247 [==============================] - 0s 2ms/step - loss: 464.3595
Epoch 6/120
247/247 [==============================] - 0s 2ms/step - loss: 462.0815
Epoch 7/120
247/247 [==============================] - 0s 2ms/step - loss: 451.9352
Epoch 8/120
247/247 [==============================] - 0s 2ms/step - loss: 459.0807
Epoch 9/120
247/247 [==============================] - 0s 2ms/step - loss: 455.7115
Epoch 10/120
247/247 [==============================] - 0s 2ms/step - loss: 452.2635
Epoch 11/120
247/247 [==============================] - 0s 2ms/step - loss: 460.0978
Epoch 12/120
247/247 [==============================] - 0s 2ms/step - loss

In [41]:
metrics_r(yr_test, model_regression.predict(Xr_test))

62/62 [==============================] - 0s 1ms/step


{'MAE': 13.995576781299915,
 'MSE': 413.8410771754459,
 'RMSE': 20.343084259163994,
 'MAPE': 0.2492361631347352,
 'R^2': 0.32692265827417855}

### Класификация

In [9]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(5,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
# в качестве функции активации используется бинарная  кроссэнтропия
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss="mse")
# verbose=None - не будет логов
model_classification_1.fit(Xc_train, yc_train, epochs=25, verbose=None)

model_classification_1.predict(Xc_test, verbose=None)[:5]

In [ ]:
yc_pred = np.around(model_classification_1.predict(Xc_test, verbose=None))

print(classification_report(yc_test, yc_pred))
confusion_matrix(yc_test, yc_pred)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1001
           1       0.50      1.00      0.67       995

    accuracy                           0.50      1996
   macro avg       0.25      0.50      0.33      1996
weighted avg       0.25      0.50      0.33      1996



array([[   0, 1001],
       [   0,  995]], dtype=int64)

In [ ]:
w0 = 1 / yc_train[yc_train==0].shape[0]
w1 = 1 / yc_train[yc_train==1].shape[0]

In [ ]:
model_classification_1 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(5,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # используем 1 нейрон и sigmoid
        tf.keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model_classification_1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="binary_crossentropy")
model_classification_1.fit(Xc_train, yc_train, epochs=25, verbose=None, class_weight={0: w0, 1: w1})
yc_pred = np.around(model_classification_1.predict(Xc_test, verbose=None))

print(classification_report(yc_test, yc_pred))
confusion_matrix(yc_test, yc_pred)

              precision    recall  f1-score   support

           0       0.85      0.02      0.04      1001
           1       0.50      1.00      0.67       995

    accuracy                           0.51      1996
   macro avg       0.67      0.51      0.36      1996
weighted avg       0.68      0.51      0.35      1996



array([[ 22, 979],
       [  4, 991]], dtype=int64)

In [ ]:
model_classification_2 = tf.keras.Sequential(
    [
        tf.keras.layers.Dense(64, activation="relu", input_shape=(5,)),
        tf.keras.layers.Dense(128, activation="relu"),
        tf.keras.layers.Dropout(0.05),
        tf.keras.layers.Dense(64, activation="relu"),
        tf.keras.layers.Dense(32, activation="relu"),
        tf.keras.layers.Dense(16, activation="relu"),
        # сначала используем 2 нейрона и softmax
        tf.keras.layers.Dense(2, activation="softmax"),
    ]
)
# в качестве функции активации используется категориальная кроссэнтропия
# используем разряженный (sparse) вариант, поскольку значения целевого признака не закодированы One-Hot кодированием
model_classification_2.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.005), loss="sparse_categorical_crossentropy")
model_classification_2.fit(Xc_train, yc_train, epochs=25, verbose=None, class_weight={0: w0, 1: w1})

model_classification_2.predict(Xc_test, verbose=None)[:5]

array([[0.5062273 , 0.49377263],
       [0.5062273 , 0.49377263],
       [0.5062273 , 0.49377263],
       [0.5062273 , 0.49377263],
       [0.5062273 , 0.49377263]], dtype=float32)

In [ ]:
# получим индексы максимального значения для каждого элемента (вложенный массив) с помощью numpy
yc_pred = [np.argmax(pred) for pred in model_classification_2.predict(Xc_test, verbose=None)]

In [ ]:
print(classification_report(yc_test, yc_pred))
confusion_matrix(yc_test, yc_pred)

              precision    recall  f1-score   support

           0       0.52      1.00      0.68      1033
           1       0.00      0.00      0.00       963

    accuracy                           0.52      1996
   macro avg       0.26      0.50      0.34      1996
weighted avg       0.27      0.52      0.35      1996



array([[1033,    0],
       [ 963,    0]], dtype=int64)

## Задание №2.
Разработать многослойный персептрон (MLP), с помощью которого можно решать задачи регрессии и классификации. Предусмотреть возможность использования таких функции активации, как sigmoid, tanh и relu; также предусмотреть возможность указать, сколько слоев нужно, сколько на каждом из них нейронов и какую функцию активации должен иметь слой. Реализовать обучение MLP методом обратного распространения ошибки; самостоятельно найти производные функций sigmoid, tanh и relu; реализовать классический градиентный спуск с возможностью указания шага.

## Дополнительное Задание №3*.
1. Самостоятельно изучить отличия работы оптимизаторов Adam и RMSProp от классического градиентного спуска.
2. Реализовать градиентный спуск с использованием указанных оптимизаторов; предусмотрите возможность использования реализованных вами оптими-заторов в Вашем персептроне.